<a href="https://colab.research.google.com/github/sgr1118/Sentence-Embedding-Using-Korean-Corpora/blob/main/03_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 전처리
- 3장에서는 임베딩 학습을 위한 한국어 데이터 전처리 과정을 다룬다. 웹 문서나 json 같은 형태의 파일을 텍스트 파일로 바꾸고 여기에 형태소 분석을 실시하는 방법을 설명한다. 형태소 분석 방법에는 국어학 전문가들이 태깅한 데이터로 학습된 모델로 분석하는 지도 학습 기법, 우리가 가진 말뭉치의 패텅을 학습한 모델을 적용하는 비지도 학습 기법 등이 있다.

## 3.1 데이터 확보
- 임베딩 학습용 말뭉치는 직접 만들거나, 웹 스크래핑을 하여 모을 수 있다. 이 교재에서는 이미 공개돼있는 말뭉치 데이터를 활용

- 임베딩에 자연어 의미를 함축하는 비법은 자연어의 통계적 패턴 정보를 통째로 임베딩에 넣는 것이다. 자연어의 의미는 해당 언어 화자들이 실제 사용하는 일상 언어에서 드러나기 때문이다. 임베딩을 만들 때 쓰는 통계 정보는 아래 3가지 있다.

### 3.1.1 한국어 위키백과
- 한국어 위키백과 raw data를 사용하려면 도커 컨테이너 bash shell을 사용하거나 pythn 상에서 코드로 불러 올 수 있다.

코드 3-1 bash 한국어 위키백과 다운로드
- git pull origin master
- bash preprocess.sh dump-raw-wiki



In [ ]:
cd '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data'

/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data


In [ ]:
# 코드 3-1 한국어 위키백과 다운로드
!pip install wikiextractor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 KB 3.0 MB/s eta 0:00:00


In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [ ]:
# !wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
# kowiki-latest-pages-articles.xml.bz2.1이 다운되는 현상이 발생하여 kowiki-latest-pages-articles.xml.bz2를 직접적으로 다운로드하여 사용

--2023-03-18 06:45:10--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.142, 2620:0:861:2:208:80:154:142
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 889750893 (849M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-articles.xml.bz2.1’

kowiki-latest-pages 100%[===================>] 848.53M  4.17MB/s    in 3m 22s  

2023-03-18 06:48:32 (4.21 MB/s) - ‘kowiki-latest-pages-articles.xml.bz2.1’ saved [889750893/889750893]



In [ ]:
# 위키익스트랙터를 사용하여 위키피디아 덤프를 파싱
!python -m wikiextractor.WikiExtractor kowiki-latest-pages-articles.xml.bz2

In [ ]:
ls

images/                                    LICENSE
install_mecab-ko_on_colab190912.sh         README.md
install_mecab-ko_on_colab_light_220429.sh  text/
kowiki-latest-pages-articles.xml.bz2


In [ ]:
# 코드 3-2 한국어 위키백과 전처리
import os
import re

os.listdir('text')

['AA', 'AB', 'AC', 'AD', 'AE', 'AF', 'AG', 'AH', 'AI']

In [ ]:
# 한국어 위키백과 전처리

from gensim.corpora import WikiCorpus, Dictionary
from gensim.utils import to_unicode
from gensim.corpora.wikicorpus import tokenize

in_f = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/kowiki-latest-pages-articles.xml.bz2'
out_f = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/precessed_wiki_ko.txt'
output = open(out_f, 'w')
wiki = WikiCorpus(in_f, tokenizer_func = tokenize, dictionary=Dictionary())
i = 0
for text in wiki.get_texts():
    output.write(bytes(' '.join(text), 'utf-8').decode('utf-8') + '\n')
    i = i + 1
    if (i % 10000 == 0):
        print('Processed ' + str(i) + ' article')
output.close()
print('Processing complete!')

Processed 10000 article
Processed 20000 article
Processed 30000 article
Processed 40000 article
Processed 50000 article
Processed 60000 article
Processed 70000 article
Processed 80000 article
Processed 90000 article
Processed 100000 article
Processed 110000 article
Processed 120000 article
Processed 130000 article
Processed 140000 article
Processed 150000 article
Processed 160000 article
Processed 170000 article
Processed 180000 article
Processed 190000 article
Processed 200000 article
Processed 210000 article
Processed 220000 article
Processed 230000 article
Processed 240000 article
Processed 250000 article
Processed 260000 article
Processed 270000 article
Processed 280000 article
Processed 290000 article
Processed 300000 article
Processed 310000 article
Processed 320000 article
Processed 330000 article
Processed 340000 article
Processed 350000 article
Processed 360000 article
Processed 370000 article
Processed 380000 article
Processed 390000 article
Processed 400000 article
Processed

- 본문 <text> 태그 안에서 필요한 내용을 추출하는 과정이 필요하다. 여기서 특수문자, 목차, 이메일 주소 등 불필요한 문자열 제거가 필요하다.

- 코드 3-2에서 tokenize 함수를 코드 3-4와 같이 수정하여 불 필요한 정보를 제거할 것이다.

In [ ]:
# 코드 3-4 사용자 정의, 한국어 위키 토크나이저

import re
from gensim.utils import to_unicode

WIKI_REMOVE_CHARS = re.compile("'+|(=+.{2,30})=+|__TOC__|(ファイル:).+|:(en|de|it|fr|es|kr|zh|no|fi):|\n", re.UNICODE)
WIKI_SPACE_CHARS = re.compile("(\\s|゙|゚|　)+", re.UNICODE)
EMAIL_PATTERN = re.compile("(^[a-zA-Z0-9_.+-]+@[a-zA-z0-9-]+\.[a-zA-Z0-9-.]+$)", re.UNICODE)
URL_PATTENR = re.compile("((ftp|http|https):\/\/)?(www.)?(?!.*(ftp|http|https|www.))[a-zA-Z0-9_-]+(\.[a-zA-Z]+)+((\/)[\w#]+)*(\/\/[a-zA-Z0-9_]+=\w+(&[a-zA-Z0-9_]+=\w+)?$)", re.UNICODE)
WIKI_REMOVE_TOKEN_CHARS = re.compile("(\\*$|:$|^파일:.+|^;)", re.UNICODE)
MULTILPE_SPACES = re.compile(' +', re.UNICODE)
def tokenizer(content, token_min_len = 2, token_max_len = 100, lower=True):
    content = re.sub(EMAIL_PATTERN, ' ', content)
    content = re.sub(URL_PATTENR, ' ', content)
    content = re.sub(WIKI_REMOVE_CHARS, ' ', content)
    content = re.sub(WIKI_SPACE_CHARS, ' ', content)
    content = re.sub(MULTILPE_SPACES, ' ', content)
    tokens = content.replace(", )", "").split(" ")
    result = []
    for token in tokens:
        if not token.startswith('_'):
            token_candidate = to_unicode(re.sub(WIKI_REMOVE_TOKEN_CHARS, '', token))
        else:
            token_candidate = ""
        if len(token_candidate) > 0:
            result.append(token_candidate)
    return result

In [ ]:
# 한국어 위키백과 최종 전처리

content = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/precessed_wiki_ko.txt'
tokenizer(content, token_min_len = 2, token_max_len = 100, lower=True)

['/content/drive/MyDrive/Book/Sentence',
 'Embedding',
 'Using',
 'Korean',
 'Corpora/data/Mecab-ko-for-Google-Colab/precessed_wiki_ko.txt']

### 3.1.2 KorQuAD
- KorQuAD는 한국어 기계 독해를 위한 데이터셋이다. LG CNS가 구축해 2018년 공개했으며 학습/데브/테스트셋을 모두 포함해 7만 79건에 이르는 데이터이다.

- KorQuAD는 구축 전 과정에서 사람들이 개입하여 검증이 철저하게 이루어졌다.

In [ ]:
# KorQuAD 데이터 셋 다운로드

!wget https://raw.githubusercontent.com/nate-parrott/squad/master/data/train-v1.1.json
!wget https://raw.githubusercontent.com/nate-parrott/squad/master/data/dev-v1.1.json

--2023-03-20 04:17:36--  https://raw.githubusercontent.com/nate-parrott/squad/master/data/train-v1.1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [text/plain]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M  64.8MB/s    in 0.4s    

2023-03-20 04:17:40 (64.8 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]

--2023-03-20 04:17:40--  https://raw.githubusercontent.com/nate-parrott/squad/master/data/dev-v1.1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6

In [ ]:
# 코드 3-8 KorQuAD 전처리 (학습용)

import json

corpus_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/KorQuAD_v1.0_train.json'
output_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/precessed_korquad_train.txt'

with open(corpus_fname) as f1, open(output_fname, 'w', encoding = 'utf-8') as f2:
    dataset_json = json.load(f1)
    dataset = dataset_json['data']
    for article in dataset:
        w_lines = []
        for paragraph in article['paragraphs']:
            w_lines.append(paragraph['context'])
            for qa in paragraph['qas']:
                q_text = qa['question']
                for a in qa['answers']:
                    a_text = a['text']
                    w_lines.append(q_text + " " + a_text)
        for line in w_lines:
            f2.writelines(line + "\n")

In [ ]:
# 코드 3-8 KorQuAD 전처리 (데브용)

import json

corpus_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/KorQuAD_v1.0_dev.json'
output_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/precessed_korquad_dev.txt'

with open(corpus_fname) as f1, open(output_fname, 'w', encoding = 'utf-8') as f2:
    dataset_json = json.load(f1)
    dataset = dataset_json['data']
    for article in dataset:
        w_lines = []
        for paragraph in article['paragraphs']:
            w_lines.append(paragraph['context'])
            for qa in paragraph['qas']:
                q_text = qa['question']
                for a in qa['answers']:
                    a_text = a['text']
                    w_lines.append(q_text + " " + a_text)
        for line in w_lines:
            f2.writelines(line + "\n")

### 3.1. 네이버 영화 리뷰 말뭉치
- 네이버 영화 리뷰 말뭉치는 영화 리뷰들을 평점과 함께 수록한 한국어 말뭉치이다. 감성 분석이나 문서 분류 태스크 수행에 제격이다. 레코드 하나는 문서(리뷰)에 대응한다. 문서 ID, 문서 내용, 레이블(긍정1, 부정0)로 구성돼 있으며 각 열은 탭 문자로 구분돼있다.

In [ ]:
# 네이버 영화 리뷰 말뭉치 다운로드

# 필요한 라이브러리 및 데이터 불러오기
import pandas as pd
#import konlpy
import re
import matplotlib.pyplot as plt
import urllib.request
import numpy as np

%matplotlib inline

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
# df = pd.read_table('ratings.txt')
# df.head()

('ratings.txt', <http.client.HTTPMessage at 0x7fd6fbba0a30>)

In [ ]:
# 코드 3-11 네이버 영화 리뷰 전처리 코드

corpus_path = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/ratings.txt'
output_fname = '/content/drive/MyDrive/Book/Sentence Embedding Using Korean Corpora/data/Mecab-ko-for-Google-Colab/process_ratings.txt'
with_label = False

with open(corpus_path, 'r', encoding = 'utf-8') as f1, open(output_fname, 'w', encoding = 'utf-8') as f2:
    next(f1) # 헤드라인 스킵
    for line in f1:
        _, sentence, label = line.strip().split('\t')
        if not sentence: continue
        if with_label:
            f2.writelines(sentence + '\u241E' + label + '\n')
        else:
            f2.writelines(sentence + '\n')

## 3.2 지도 학습 기반 형태소 분석
- 품질 좋은 임베딩을 만들기 위해서는 문장이나 단어의 경계를 컴퓨터에 알려줘야 한다. 그렇지 않으면 어휘 집합에 속한 단어 수가 기하급수적으로 늘어나서 연산의 비효율이 발생한다. 특히 한국어는 조사와 어미가 발달한 교착어이기 때문에 이런 처리를 더 꼼꼼하게 해야한다.

- 예를 들어 동사 '가다'는 '가겠다', '가더라'라는 활용이 가능하다. 이런 활용형을 모두 어휘 집합에 넣을 수 있지만 새로운 활용형이 발생할 때마다 계속 늘려야 한다는 단점이 존재한다. 이런 문제를 해소하고자 형태소 분석 기법을 사용한다.

- 오픈소스인 Mecab을 사용하여 형태소 분석기로 사용할 수 있다. Mecab을 사용하용 활용형 어휘를 분석하면 아래와 같다.

- 가겠다 > 가, 겠, 다
- 가더라 > 가, 더라

- 이런 형태소 분석 덕분에 어휘 집합 추가 없이 처리할 수 있는 것이다.

- 교착어인 한국어는 한정된 종류의 조사와 어미를 자주 이용하기 때문에 각각에 대응하는 명사, 용언(형용사, 동사) 어간만 어휘 집합에 추가하면 취급 단어 개수를 꽤 줄일 수 있다. 형태소 분석기를 잘 사용하면 NLP의 효율성을 높일 수 있다는 것이다.

- 이 파트에서는 지도 학습 기반의 형태소 분석을 수행한다. 지도 학습이란 정답이 주어진 데이터를 모델에 넣어 패턴을 학습하게하는 방법이다. 또한 이 파트에서 설명되는 형태소 분석기들은 언어학 전문가들이 태깅한 형태소 분석 말뭉치로부터 학습된 지도 학습 기반 모델들이다.

- 태깅이란 아래처럼 모델 입력과 출력 쌍을 만드는 작업을 가리킨다. 이 모델들은 문자열이 주어질 때 사람이 알려준 정답 패턴에 최대한 가깝게 토크나이즈한다.

- 입력 : 아버지가방에들어가신다
- 출력 : 아버지, 가, 방, 에, 들어가, 신다.

### 3.2.1 KoNLPy 사용법
- KoNLPy는 Mecab, 꼬꼬마, 한나눔, Okt, 코모란 등 5개 오픈소스 형태소 분석기를 파이썬 화녕에서 사용할 수 있도록한다.

In [ ]:
# KoNLPy 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

yes
checking whether g++ accepts -g... yes
checking dependency style of g++... none
checking how to run the C preprocessor... gcc -E
checking for grep that handles long lines and -e... /usr/bin/grep
checking for egrep... /usr/bin/grep -E
checking whether gcc needs -traditional... no
checking whether make sets $(MAKE)... (cached) yes
checking build system type... x86_64-unknown-linux-gnu
checking host system type... x86_64-unknown-linux-gnu
checking how to print strings... printf
checking for a sed that does not truncate output... /usr/bin/sed
checking for fgrep... /usr/bin/grep -F
checking for ld used by gcc... /usr/bin/ld
checking if the linker (/usr/bin/ld) is GNU ld... yes
checking for BSD- or MS-compatible name lister (nm)... /usr/bin/nm -B
checking the name lister (/usr/bin/nm -B) interface... BSD nm
checking whether ln -s works... yes
checking the maximum length of command line arguments... 1572864
checking whether the shell understands some XSI constructs... yes
checking whether

In [ ]:
# 코드 3-14 Mecab 활용

from konlpy.tag import Mecab
tokenizer = Mecab()
tokenizer.morphs('아버지가방에들어가신다')

['아버지', '가', '방', '에', '들어가', '신다']

In [ ]:
# 코드 3-15 Mecab 품사 정보 확인
tokenizer.pos('아버지가방에들어가신다')

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('신다', 'EP+EC')]

In [ ]:
# 코드 3-16 get_tokenizer 함수 만들기
# 5개 토크나이저 비교를 위함

from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

def get_tokenizer(tokenizer_name):
    if tokenizer_name == 'komoran':
        tokenizer = Komoran()
    elif tokenizer_name == 'okt':
        tokenizer = Okt()
    elif tokenizer_name == 'mecab':
        tokenizer = Mecab()
    elif tokenizer_name == 'hannanum':
        tokenizer = Hannanum()
    elif tokenizer_name == 'kkma':
        tokenizer = Kkma()
    else:
        tokenizer = Mecab()
    return tokenizer

# 코모란 사용 예시
tokenizer = get_tokenizer('komoran')
print(tokenizer.morphs('아버지가방에들어가신다'))
print(tokenizer.pos('아버지가방에들어가신다'))

['아버지', '가방', '에', '들어가', '시', 'ㄴ다']
[('아버지', 'NNG'), ('가방', 'NNP'), ('에', 'JKB'), ('들어가', 'VV'), ('시', 'EP'), ('ㄴ다', 'EC')]


In [ ]:
# Okt 사용 결과
tokenizer = get_tokenizer('okt')
print(tokenizer.morphs('아버지가방에들어가신다'))
print(tokenizer.pos('아버지가방에들어가신다'))

['아버지', '가방', '에', '들어가신다']
[('아버지', 'Noun'), ('가방', 'Noun'), ('에', 'Josa'), ('들어가신다', 'Verb')]


In [ ]:
# 코모란 사용 예시
tokenizer = get_tokenizer('komoran')
tokenizer.morphs('아버지가방에들어가신다')
tokenizer.pos('아버지가방에들어가신다')

### 3.2.2 KoNLPy 내 분석기별 성능 차이 분석

![](https://konlpy-ko.readthedocs.io/ko/v0.4.3/_images/time.png)
<center>문자 개수 대비 실행 시간</center>

- Mecab이 다른 분석기 대비 상대적으로 속도가 빠르다는 것을 볼 수 있다.

### 3.2.3 Khaiii 사용법
- Khaiii는 카카오가 2018년 말 공개한 오픈소스 한국어 형태소 분석기이다. 국립국어원이 구축한 세종 코퍼스를 이용해 CNN모델을 적용해 학습했다. Khaiii의 아키텍처 개요는 아래 그림과 같다.

![](https://t1.daumcdn.net/thumb/R1280x0/?fname=http://t1.daumcdn.net/brunch/service/user/1oU7/image/DXlTnCNYfeYzWIR4kN428VouYKQ.png)
<center>Khaiii 아케텍처</center>

- 입력 문장을 문자 단위로 읽고 컨볼루션 필터가 이 문자들을 나눠가면서 정보를 추출한다. 이 네트워크의 말단 레이어에서는 이렇게 모은 정보들을 종합해 형태소의 경계와 품사 태그를 예측한다. GPU없이도 형태소 분석이 가능하며 실행 속도도 빠르다고 한다.